In [ ]:
import numpy as np
import pandas as pd
from timeit import timeit

# Enhancing performance

We first look at native code compilation. Here we show 3 common methods for doing this using `numba` JIT compilation, `cython` AOT compilation, and direct wrapping of C++ code using `pybind11`. In general, `numba` is the simplest to use, while you have the most flexibility with `pybind11`. Which approach gives the best performance generally requires some experimentation.

Then we review common methods for concurrent execution of embarrassingly parallel code using `multiprocessing`, `concurrent.futures` and `joblib`. Comparison of performance using processes and threads is made, with a brief explanation of the Global Interpreter Lock (GIL).

More details for each of the libraries used to improve performance is provided in the course notebooks.

## Python

In [ ]:
def cdist(xs, ys):
    """Returns pairwise distance between row vectors in xs and ys.
    
    xs has shape (m, p)
    ys has shape (n, p)
    
    Return value has shape (m, n)    
    """
    
    m, p = xs.shape
    n, p = ys.shape
    
    res = np.empty((m, n))
    for i in range(m):
        for j in range(n):
            res[i, j] = np.sqrt(np.sum((ys[j] - xs[i])**2))
    return res

### Sanity check

In [ ]:
xs = np.arange(6).reshape(-1,2).astype('float')
ys = np.arange(4).reshape(-1, 2).astype('float')
zs = cdist(xs, ys)

In [ ]:
zs

In [ ]:
%timeit -r 3 -n 10 cdist(xs, ys)

In [ ]:
m = 1000
n = 1000
p = 100

X = np.random.random((m, p))
Y = np.random.random((n, p))

In [ ]:
%%time

Z = cdist(X, Y)

In [ ]:
t0 = timeit(lambda : cdist(X, Y), number=1)

## Using `numba`

In [ ]:
from numba import jit, njit

In [ ]:
@njit
def cdist_numba(xs, ys):
    """Returns pairwise distance between row vectors in xs and ys.
    
    xs has shape (m, p)
    ys has shape (n, p)
    
    Return value has shape (m, n)    
    """
    
    m, p = xs.shape
    n, p = ys.shape
    
    res = np.empty((m, n))
    for i in range(m):
        for j in range(n):
            res[i, j] = np.sqrt(np.sum((ys[j] - xs[i])**2))
    return res

Check

In [ ]:
assert(np.allclose(cdist(xs, ys), cdist_numba(xs, ys)))

In [ ]:
%%time

Z = cdist_numba(X, Y)

In [ ]:
t_numba = timeit(lambda : cdist_numba(X, Y), number=1)

### Unrolling

We can help `numba` by unrolling the code.

In [ ]:
@njit
def cdist_numba1(xs, ys):
    """Returns pairwise distance between row vectors in xs and ys.
    
    xs has shape (m, p)
    ys has shape (n, p)
    
    Return value has shape (m, n)    
    """
    
    m, p = xs.shape
    n, p = ys.shape
    
    res = np.empty((m, n))
    for i in range(m):
        for j in range(n):
            s = 0
            for k in range(p):
                s += (ys[j,k] - xs[i,k])**2
            res[i, j] = np.sqrt(s)
    return res

Check

In [ ]:
assert(np.allclose(cdist(xs, ys), cdist_numba1(xs, ys)))

In [ ]:
%%time

Z = cdist_numba1(X, Y)

In [ ]:
t_numba1 = timeit(lambda : cdist_numba1(X, Y), number=1)

## Using `cython`

In [ ]:
%load_ext cython

In [ ]:
%%cython -a

import numpy as np

def cdist_cython(xs, ys):
    """Returns pairwise distance between row vectors in xs and ys.
    
    xs has shape (m, p)
    ys has shape (n, p)
    
    Return value has shape (m, n)    
    """
    
    m, p = xs.shape
    n, p = ys.shape
    
    res = np.empty((m, n))
    for i in range(m):
        for j in range(n):
            res[i, j] = np.sqrt(np.sum((ys[j] - xs[i])**2))
    return res

Check

In [ ]:
assert(np.allclose(cdist(xs, ys), cdist_cython(xs, ys)))

In [ ]:
%%time

Z = cdist_cython(X, Y)

In [ ]:
t_cython = timeit(lambda : cdist_cython(X, Y), number=1)

In [ ]:
%%cython -a

import cython
import numpy as np
from libc.math cimport sqrt, pow

@cython.boundscheck(False)
@cython.wraparound(False)
def cdist_cython1(double[:, :] xs, double[:, :] ys):
    """Returns pairwise distance between row vectors in xs and ys.
    
    xs has shape (m, p)
    ys has shape (n, p)
    
    Return value has shape (m, n)    
    """
    
    cdef int m, n, p
    
    m = xs.shape[0]
    n = ys.shape[0]
    p = xs.shape[1]
    
    cdef double[:, :] res = np.empty((m, n))
    
    cdef int i, j
    
    cdef double s
    for i in range(m):
        for j in range(n):
            s = 0.0
            for k in range(p):
                s += pow(ys[j,k] - xs[i,k], 2)                
            res[i, j] = sqrt(s)
    return res

Check

In [ ]:
assert(np.allclose(cdist(xs, ys), cdist_cython(xs, ys)))

In [ ]:
%%time

Z = cdist_cython1(X, Y)

In [ ]:
t_cython1 = timeit(lambda : cdist_cython1(X, Y), number=1)

In [ ]:
perf = pd.DataFrame(dict(
    methods = ['python', 'numba', 'numba1',  'cython', 'cython1'],
    times = [t0, t_numba, t_numba1, t_cython, t_cython1],
))

In [ ]:
perf['speed-up'] = np.around(perf['times'][0]/perf['times'], 1)
perf

## Using multiple cores

The standard implementation of Python uses a Global Interpreter Lock (GIL). This means that only one thread can be run at any one time, and multiple threads work by time-slicing. Hence multi-threaded code with lots of latency can result in speed-ups, but multi-threaded code which is computationally intensive will not see any speed-up. For numerically intensive code, parallel code needs to be run in separate processes to see speed-ups.

First we see how to split the computation into pieces using a loop.

In [ ]:
xs

In [ ]:
ys

In [ ]:
cdist(xs, ys)

In [ ]:
res = np.concatenate([cdist(x, ys) for x in np.split(xs, 3, 0)])
res

In [ ]:
%%time

Z = cdist(X, Y)

### Using `multiprocessing`

In [ ]:
from multiprocessing import Pool

In [ ]:
%%time

with Pool(processes=4) as p:
    Z1 = p.starmap(cdist, [(X_, Y) for X_ in np.split(X, 100, 0)])
    Z1 = np.concatenate(Z1)

Check

In [ ]:
np.testing.assert_allclose(Z, Z1)

### Using `concurrent.futures

In [ ]:
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor

In [ ]:
def cdist_(args):
    return cdist(*args)

In [ ]:
%%time

with ProcessPoolExecutor(max_workers=4) as pool:
    Z2 = list(pool.map(cdist_, [(X_, Y) for X_ in np.split(X, 100, 0)]))
    Z2 = np.concatenate(Z2)

Check

In [ ]:
np.testing.assert_allclose(Z, Z2)

### Using `joblib`

`joblib` provides parallel processing using a comprehension syntax.

In [ ]:
from joblib import Parallel, delayed

In [ ]:
%%time

Z3 = Parallel(n_jobs=4)(delayed(cdist)(X_, Y) for X_ in np.split(X, 100, 0))
Z3 = np.concatenate(Z3)

Check

In [ ]:
np.testing.assert_allclose(Z, Z3)

### Using threads

Note that there is no gain with using multiple threads for computationally intensive tasks because of the GIL.

In [ ]:
%%time

with ThreadPoolExecutor(max_workers=4) as pool:
    Z4 = list(pool.map(cdist_, [(X_, Y) for X_ in np.split(X, 100, 0)]))
    Z4 = np.concatenate(Z4)

Check

In [ ]:
np.testing.assert_allclose(Z, Z4)